# Ejercicios del libro

## Funciones definidas por el usuario

### Spark SQL UDFs
Estamos creando una función de calcula el cubo de números reales

In [0]:
%scala
val cubed = (s: Long) => {
 s * s * s
}
// Register UDF
spark.udf.register("cubed", cubed)
// Create temporary view
spark.range(1, 9).createOrReplaceTempView("udf_test")

cubed: Long => Long = $Lambda$8075/1538419612@110cc916

In [0]:
%scala
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

+---+--------+
 id|id_cubed|
+---+--------+
 1| 1|
 2| 8|
 3| 27|
 4| 64|
 5| 125|
 6| 216|
 7| 343|
 8| 512|
+---+--------+

## Consultas con Spark SQL Shell, Beeline y Tableau

### Utilizando la Spark SQL Shell

#### Crear una tabla

In [0]:
%sql
CREATE TABLE people (name STRING, age int);

In [0]:
%sql
INSERT INTO people VALUES ("Michael", NULL);
INSERT INTO people VALUES ("Andy", 30);
INSERT INTO people VALUES ("Samantha", 19);

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql 
show tables

database,tableName,isTemporary
default,people,false
,udf_test,true


In [0]:
%sql
select * from people

name,age
Samantha,19
Andy,30
Michael,null


### Trabajando con Beeline
Es una herramienta para ejecutar consultas de HiveQL. Es Hive, pero para versiones superiores de Spark.

**Para trabajar con Beerline hay que hacerlo desde Clouder ejecutando el comando ```Beerline```**. Luego hay que conectarse al servidor Thrift. Para ello, se ejecuta el siguiente comando dentro de Beerline ```!connect jdbc:hive2://localhost:10000```, se introduce mi correo como usuario y 'blanck' como contraseña.

Creamos la tabla people y ejecutamos consultas del mismo modo que se hace en Hive.

**Trabajando con Beerline y el servidor Thrift desde una consola**

Para inicializar Beerline: ```./bin/beeline```

Para conectar con Thrift: ```!connect jdbc:hive2://localhost:10000```

### Trabajando con Tableau
Mediante tableau también nos podemos conectar al servidir Thrift. Para ello habrá que inicualizar el servidor con ```./sbin/start-thriftserver.sh``` y si aún no ha iniciado el driver de Spark, ejecutar ```/sbin/start-all.sh```. Luego conectamos con el servidir mediante Tableau como indican las páginas 148-149 y podremos acceder a las tablas creadas mediante Beerline o Hive. Por último, habrá que detener el servidor Thrift con ```./sbin/stop-thriftserver.sh```.

## Fuentes de datos externas
Se verá cómo conectar Spark SQL para trabajar con bases de datos externas como bases de datos JDBC y SQL.

### Bases de datos JDBC y SQL
Spark incluye una API que permite leer datos de otras bases de datos mediante JDBC. Esto simplifica las consultas debido a que las devuelve en forma de DataFrame. 

Para inicializar la Driver de JDBC se ejecuta el siguiente comando ```./bin/spark-shell --driver-class-path $database.jar --jars $database.jar```

#### La importancia de particionar
Cuando hay grandes volúmenes de datos siempre particionar ralentiza los tiempos de ejecución. Es adecuado hacer un número de particiones proporcional al número de nodos. Habrá que especificar el límite inferior y el superior de los valores de la columna de partición, que es como un id.

### PostgreSQL

Cómo cargar y guardar una base de datos PostgreSQL.

In [0]:
%scala
// Read Option 1: Loading data from a JDBC source using load method
val jdbcDF1 = spark
 .read
 .format("jdbc")
 .option("url", "jdbc:postgresql:[DBSERVER]")
 .option("dbtable", "[SCHEMA].[TABLENAME]")
 .option("user", "[USERNAME]")
 .option("password", "[PASSWORD]")
 .load()
// Read Option 2: Loading data from a JDBC source using jdbc method
// Create connection properties
import java.util.Properties
val cxnProp = new Properties()
cxnProp.put("user", "[USERNAME]")
cxnProp.put("password", "[PASSWORD]")
// Load data using the connection properties
val jdbcDF2 = spark
 .read
 .jdbc("jdbc:postgresql:[DBSERVER]", "[SCHEMA].[TABLENAME]", cxnProp)
// Write Option 1: Saving data to a JDBC source using save method
jdbcDF1
 .write
 .format("jdbc")
 .option("url", "jdbc:postgresql:[DBSERVER]")
 .option("dbtable", "[SCHEMA].[TABLENAME]")
 .option("user", "[USERNAME]")
 .option("password", "[PASSWORD]")
 .save()
// Write Option 2: Saving data to a JDBC source using jdbc method
jdbcDF2.write
 .jdbc(s"jdbc:postgresql:[DBSERVER]", "[SCHEMA].[TABLENAME]", cxnProp)

### MySQL

In [0]:
%scala
// In Scala
// Loading data from a JDBC source using load 
val jdbcDF = spark
 .read
 .format("jdbc")
 .option("url", "jdbc:mysql://[DBSERVER]:3306/[DATABASE]")
 .option("driver", "com.mysql.jdbc.Driver")
 .option("dbtable", "[TABLENAME]")
 .option("user", "[USERNAME]")
 .option("password", "[PASSWORD]")
 .load()
// Saving data to a JDBC source using save 
jdbcDF
 .write
 .format("jdbc")
 .option("url", "jdbc:mysql://[DBSERVER]:3306/[DATABASE]")
 .option("driver", "com.mysql.jdbc.Driver")
 .option("dbtable", "[TABLENAME]")
 .option("user", "[USERNAME]")
 .option("password", "[PASSWORD]")
 .save()


### Azure Cosmos DB

In [0]:
%scala
// In Scala
// Import necessary libraries
import com.microsoft.azure.cosmosdb.spark.schema._
import com.microsoft.azure.cosmosdb.spark._
import com.microsoft.azure.cosmosdb.spark.config.Config
// Loading data from Azure Cosmos DB
// Configure connection to your collection
val query = "SELECT c.colA, c.coln FROM c WHERE c.origin = 'SEA'"
val readConfig = Config(Map(
 "Endpoint" -> "https://[ACCOUNT].documents.azure.com:443/",
 "Masterkey" -> "[MASTER KEY]",
 "Database" -> "[DATABASE]",
 "PreferredRegions" -> "Central US;East US2;",
 "Collection" -> "[COLLECTION]",
 "SamplingRatio" -> "1.0",
 "query_custom" -> query
))
// Connect via azure-cosmosdb-spark to create Spark DataFrame
val df = spark.read.cosmosDB(readConfig)
df.count
// Saving data to Azure Cosmos DB
// Configure connection to the sink collection
val writeConfig = Config(Map(
 "Endpoint" -> "https://[ACCOUNT].documents.azure.com:443/",
 "Masterkey" -> "[MASTER KEY]",
 "Database" -> "[DATABASE]",
 "PreferredRegions" -> "Central US;East US2;",
 "Collection" -> "[COLLECTION]",
 "WritingBatchSize" -> "100"
))

### MS SQL Server

In [0]:
%scala
// In Scala
// Loading data from a JDBC source
// Configure jdbcUrl
val jdbcUrl = "jdbc:sqlserver://[DBSERVER]:1433;database=[DATABASE]"
// Create a Properties() object to hold the parameters. 
// Note, you can create the JDBC URL without passing in the
// user/password parameters directly.
val cxnProp = new Properties()
cxnProp.put("user", "[USERNAME]")
cxnProp.put("password", "[PASSWORD]")
cxnProp.put("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
// Load data using the connection properties
val jdbcDF = spark.read.jdbc(jdbcUrl, "[TABLENAME]", cxnProp)
// Saving data to a JDBC source
jdbcDF.write.jdbc(jdbcUrl, "[TABLENAME]", cxnProp)

### Otras fuentes de datos externas
Estas son otro tipo de fuentes de datos muy populares a las que Apache Spark no se puede conectar:
- Apache Cassandra
- Snowflake
- MongoDB

## Funciones de orden superior en DataFrames y Spark SQL
Hay dos soluciones para abordar la lectura o la manupulación de datos complejos:
- Dividir en filas individulaes, ejecutar la consulta y volver a juntar 
- Crear una función definida por el usuario

### Opción 1: Explotar y recopilar

In [0]:
%sql
SELECT id, collect_list(value + 1) AS values
FROM (SELECT id, EXPLODE(values) AS value
 FROM table) x
GROUP BY id

Mientras que ```collect_list()``` devuelve una lista de objetos con duplicados, la instrucción GROUP BY requiere operaciones aleatorias, lo que significa que el orden de la matriz recopilada no es necesariamente el mismo que el de la matriz original.

### Opción 2: Función definida por el usuario

In [0]:
%scala
spark.sql("SELECT id, plusOneInt(values) AS values FROM table").show()

### Funciones integradas para tipos de datos complejos
En lugar de utilizar estas técnicas más costosas, es posible utilizar algunas de las funciones integradas para tipos de datos complejos.

# PONER TABLAS páginas 163-165

### Funciones de orden superior
Además de las funciones integradas mencionadas anteriormente, existen funciones de orden superior que toman funciones lambda anónimas como argumentos.

In [0]:
%sql
transform(values, value -> lambda expression)

La función transform() toma una matriz (valores) y una función anónima (expresión lambda) como entrada. La función crea de forma transparente una nueva matriz aplicando la función anónima a cada elemento y luego asignando el resultado a la matriz de salida.

In [0]:
%scala
// Create DataFrame with two rows of two arrays (tempc1, tempc2)
val t1 = Array(35, 36, 32, 30, 40, 42, 38)
val t2 = Array(31, 32, 34, 55, 56)
val tC = Seq(t1, t2).toDF("celsius")
tC.createOrReplaceTempView("tC")
// Show the DataFrame
tC.show()

+--------------------+
 celsius|
+--------------------+
[35, 36, 32, 30, ...|
[31, 32, 34, 55, 56]|
+--------------------+

t1: Array[Int] = Array(35, 36, 32, 30, 40, 42, 38)
t2: Array[Int] = Array(31, 32, 34, 55, 56)
tC: org.apache.spark.sql.DataFrame = [celsius: array<int>]

**Funciones de orden superior que se pueden ejecutar con la base de datos anterior**

- transform()
```transform(array<T>, function<T, U>): array<U>```
La función transform() produce una matriz al aplicar una función a cada elemento de la matriz de entrada (similar a una función map()):

In [0]:
%scala
spark.sql("""
SELECT celsius, 
 transform(celsius, t -> ((t * 9) div 5) + 32) as fahrenheit 
 FROM tC
""").show()

+--------------------+--------------------+
 celsius| fahrenheit|
+--------------------+--------------------+
[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+

- filter()

```filter(array<T>, function<T, Boolean>): array<T>```

La función filter () produce una matriz que consta solo de los elementos de la matriz de entrada para los que una función booleana es verdadera.

In [0]:
%scala
spark.sql("""
SELECT celsius, 
 filter(celsius, t -> t > 38) as high 
 FROM tC
""").show()

+--------------------+--------+
 celsius| high|
+--------------------+--------+
[35, 36, 32, 30, ...|[40, 42]|
[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+

- exists()

```exists(array<T>, function<T, V, Boolean>): Boolean```

La función existe() devuelve verdadero si la función booleana se cumple para cualquier elemento en la matriz de entrada.

In [0]:
%scala
spark.sql("""
SELECT celsius, 
 exists(celsius, t -> t = 38) as threshold
 FROM tC
""").show()

+--------------------+---------+
 celsius|threshold|
+--------------------+---------+
[35, 36, 32, 30, ...| true|
[31, 32, 34, 55, 56]| false|
+--------------------+---------+

- reduce()

```reduce(array<T>, B, function<B, T, B>, function<B, R>)```

La función reduce () reduce los elementos de la matriz a un solo valor

In [0]:
%scala
spark.sql("""
SELECT celsius, 
 reduce(
 celsius, 
 0, 
 (t, acc) -> t + acc, 
 acc -> (acc div size(celsius) * 9 div 5) + 32
 ) as avgFahrenheit 
 FROM tC
""").show()

+--------------------+-------------+
 celsius|avgFahrenheit|
+--------------------+-------------+
[35, 36, 32, 30, ...| 96|
[31, 32, 34, 55, 56]| 105|
+--------------------+-------------+

## Operaciones comunes en DataFrames y Spark SQL

- Funciones agregadas
- Funciones de colección
- Funciones de fecha y hora
- Funciones no agregadas
- Funciones matemáticas
- Funciones de clasificación
- Funciones agregadas
- Funciones de cadena
- Funciones UDF
- Funciones de ventana

In [0]:
%scala
import org.apache.spark.sql.functions._
// Set file paths
val delaysPath =
 "/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
val airportsPath =
 "/databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt"
// Obtain airports data set
val airports = spark.read
 .option("header", "true")
 .option("inferschema", "true")
 .option("delimiter", "\t")
 .csv(airportsPath)
airports.createOrReplaceTempView("airports_na")
// Obtain departure Delays data set
val delays = spark.read
 .option("header","true")
 .csv(delaysPath)
 .withColumn("delay", expr("CAST(delay as INT) as delay"))
 .withColumn("distance", expr("CAST(distance as INT) as distance"))
delays.createOrReplaceTempView("departureDelays")
// Create temporary small table
val foo = delays.filter(
 expr("""origin == 'SEA' AND destination == 'SFO' AND 
 date like '01010%' AND delay > 0"""))
foo.createOrReplaceTempView("foo")


import org.apache.spark.sql.functions._
delaysPath: String = /databricks-datasets/learning-spark-v2/flights/departuredelays.csv
airportsPath: String = /databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt
airports: org.apache.spark.sql.DataFrame = [City: string, State: string ... 2 more fields]
delays: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 3 more fields]
foo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, delay: int ... 3 more fields]

In [0]:
%scala
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

+-----------+-----+-------+----+
 City|State|Country|IATA|
+-----------+-----+-------+----+
 Abbotsford| BC| Canada| YXX|
 Aberdeen| SD| USA| ABR|
 Abilene| TX| USA| ABI|
 Akron| OH| USA| CAK|
 Alamosa| CO| USA| ALS|
 Albany| GA| USA| ABY|
 Albany| NY| USA| ALB|
Albuquerque| NM| USA| ABQ|
 Alexandria| LA| USA| AEX|
 Allentown| PA| USA| ABE|
+-----------+-----+-------+----+

In [0]:
%scala
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01011245| 6| 602| ABE| ATL|
01020600| -8| 369| ABE| DTW|
01021245| -2| 602| ABE| ATL|
01020605| -4| 602| ABE| ATL|
01031245| -4| 602| ABE| ATL|
01030605| 0| 602| ABE| ATL|
01041243| 10| 602| ABE| ATL|
01040605| 28| 602| ABE| ATL|
01051245| 88| 602| ABE| ATL|
01050605| 9| 602| ABE| ATL|
+--------+-----+--------+------+-----------+

In [0]:
%scala
spark.sql("SELECT * FROM foo").show()

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
+--------+-----+--------+------+-----------+

### Unions

In [0]:
%scala
val bar = delays.union(foo)
bar.createOrReplaceTempView("bar")
bar.filter(expr("""origin == 'SEA' AND destination == 'SFO'
AND date LIKE '01010%' AND delay > 0""")).show()

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
+--------+-----+--------+------+-----------+

bar: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, delay: int ... 3 more fields]

### Joins

In [0]:
%scala
foo.join(
 airports.as('air),
 $"air.IATA" === $"origin"
).select("City", "State", "date", "delay", "distance", "destination").show()

+-------+-----+--------+-----+--------+-----------+
 City|State| date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
Seattle| WA|01010710| 31| 590| SFO|
Seattle| WA|01010955| 104| 590| SFO|
Seattle| WA|01010730| 5| 590| SFO|
+-------+-----+--------+-----+--------+-----------+

### Windowing
Una función de ventana usa valores de las filas en una ventana (un rango de filas de entrada) para el marco de datos de los aeropuertos:
devuelve un conjunto de valores, normalmente en forma de otra fila. Con funciones de ventana, es posible operar en un grupo de filas y aún devolver un valor único para cada fila de entrada. Una de ellas es ```dense_rank()```.

## TABLA página 173

In [0]:
%sql 
DROP TABLE IF EXISTS departureDelaysWindow;

In [0]:
%sql 
CREATE TABLE departureDelaysWindow AS
SELECT origin, destination, SUM(delay) AS TotalDelays
 FROM departureDelays
WHERE origin IN ('SEA', 'SFO', 'JFK')
 AND destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL')
GROUP BY origin, destination;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM departureDelaysWindow

origin,destination,TotalDelays
JFK,ORD,5608
JFK,SFO,35619
JFK,DEN,4315
JFK,ATL,12141
JFK,SEA,7856
JFK,LAX,35755
SEA,LAX,9359
SFO,ORD,27412
SFO,DEN,18688
SFO,SEA,17080


Si queremos ordenar resultados

In [0]:
%sql
SELECT origin, destination, SUM(TotalDelays) AS TotalDelays
FROM departureDelaysWindow
WHERE origin = '[ORIGIN]'
GROUP BY origin, destination
ORDER BY SUM(TotalDelays) DESC
LIMIT 3

Pero es mejor si se utiliza una window function

In [0]:
%scala
spark.sql("""
SELECT origin, destination, TotalDelays, rank 
 FROM ( 
 SELECT origin, destination, TotalDelays, dense_rank() 
 OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank 
 FROM departureDelaysWindow
 ) t 
 WHERE rank <= 3
""").show()

+------+-----------+-----------+----+
origin|destination|TotalDelays|rank|
+------+-----------+-----------+----+
 JFK| LAX| 35755| 1|
 JFK| SFO| 35619| 2|
 JFK| ATL| 12141| 3|
 SEA| SFO| 22293| 1|
 SEA| DEN| 13645| 2|
 SEA| ORD| 10041| 3|
 SFO| LAX| 40798| 1|
 SFO| ORD| 27412| 2|
 SFO| JFK| 24100| 3|
+------+-----------+-----------+----+

Con el uso de la función de ventana ```dense_rank()```, podemos determinar rápidamente que los destinos con los perores retrasos son:

- Seattle (SEA): San Francisco (SFO), Denver (DEN), and Chicago (ORD)
- San Francisco (SFO): Los Angeles (LAX), Chicago (ORD), and New York (JFK)
- New York (JFK): Los Angeles (LAX), San Francisco (SFO), and Atlanta (ATL)

### Modificaciones
Otra operación común es realizar modificaciones al DataFrame. Los DataFrames en sí mismos son inmutables, pero se pueden modificar a través de operaciones que crean nuevos DataFrames.

In [0]:
%scala
foo.show()

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
+--------+-----+--------+------+-----------+

#### Añadiendo nuevas columnas

In [0]:
%scala
import org.apache.spark.sql.functions.expr
val foo2 = foo.withColumn(
 "status",
 expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END")
 )

import org.apache.spark.sql.functions.expr
foo2: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 4 more fields]

In [0]:
%scala
foo2.show()

+--------+-----+--------+------+-----------+-------+
 date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
01010710| 31| 590| SEA| SFO|Delayed|
01010955| 104| 590| SEA| SFO|Delayed|
01010730| 5| 590| SEA| SFO|On-time|
+--------+-----+--------+------+-----------+-------+

#### Eliminando columnas

In [0]:
%scala
val foo3 = foo2.drop("delay")
foo3.show()

+--------+--------+------+-----------+-------+
 date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
01010710| 590| SEA| SFO|Delayed|
01010955| 590| SEA| SFO|Delayed|
01010730| 590| SEA| SFO|On-time|
+--------+--------+------+-----------+-------+

foo3: org.apache.spark.sql.DataFrame = [date: string, distance: int ... 3 more fields]

#### Renombrando columnas

In [0]:
%scala
val foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()%md
#### Pivoting
Consiste en intercambiar las columnas por las filas. 

+--------+--------+------+-----------+-------------+
 date|distance|origin|destination|flight_status|
+--------+--------+------+-----------+-------------+
01010710| 590| SEA| SFO| Delayed|
01010955| 590| SEA| SFO| Delayed|
01010730| 590| SEA| SFO| On-time|
+--------+--------+------+-----------+-------------+

foo4: org.apache.spark.sql.DataFrame = [date: string, distance: int ... 3 more fields]

#### Pivoting
Consiste en intercambiar las columnas por las filas.

In [0]:
%sql
SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
 FROM departureDelays
WHERE origin = 'SEA'

destination,month,delay
ORD,1,92
JFK,1,-7
DFW,1,-5
MIA,1,-3
DFW,1,-3
DFW,1,1
ORD,1,-10
DFW,1,-6
DFW,1,-2
ORD,1,-3
